<a href="https://colab.research.google.com/github/Drishty06/PCOS-Sakhi/blob/main/PCOS_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PCOS prediction**


### import libraries


In [4]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

from sklearn.metrics import mean_squared_error, accuracy_score, confusion_matrix


### loaded the csv file (local file)


In [5]:
from google.colab import files
data_to_load = files.upload()


### mounted google drive

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/pcos.csv')

In [8]:
df.head(10)


,Sl No,Patient File No,PCOS (Y/N),Age (yrs),Pregnant(Y/N),No of aborptions,Bloated,facial hair,chest hair,difficult to loose weight,...,Height(Cm),BMI,Blood Group,Pulse rate(bpm),Cycle(months),Cycle length(days),Marriage Status (Yrs),Hip(inch),Waist(inch),Waist/Hip Ratio
0,1,10001,0,28,0,0,0,0,0,0,...,152.0,19.300000,15,78,1,5,7.0,36,30,0.833333
1,2,10002,0,36,1,0,0,0,0,0,...,161.5,24.921163,15,74,1,5,11.0,38,32,0.842105
2,3,10003,1,33,1,0,1,1,1,1,...,165.0,25.270891,11,72,2,5,10.0,40,36,0.900000
3,4,10004,0,37,0,0,0,0,0,0,...,148.0,29.674945,13,72,1,5,4.0,42,36,0.857143
4,5,10005,0,25,1,0,0,0,0,0,...,161.0,20.060954,11,72,1,5,1.0,37,30,0.810811
5,6,10006,0,36,1,0,0,0,0,0,...,165.0,27.217631,15,78,1,5,8.0,44,38,0.863636
6,7,10007,0,34,0,0,0,0,0,0,...,156.0,26.298488,11,72,1,5,2.0,39,33,0.846154
7,8,10008,0,33,1,2,0,0,0,0,...,159.0,23.139907,13,72,1,5,13.0,44,38,0.863636
8,9,10009,0,32,0,1,0,0,0,0,...,158.0,16.023073,11,72,1,5,8.0,39,35,0.897436
9,10,10010,0,36,0,0,0,0,0,0,...,150.0,23.111111,15,80,1,2,4.0,40,38,0.950000


### info of dataset


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541 entries, 0 to 540
Data columns (total 31 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Sl No                      541 non-null    int64  
 1   Patient File No            541 non-null    int64  
 2   PCOS (Y/N)                 541 non-null    int64  
 3    Age (yrs)                 541 non-null    int64  
 4   Pregnant(Y/N)              541 non-null    int64  
 5   No of aborptions           541 non-null    int64  
 6   Bloated                    541 non-null    int64  
 7   facial hair                541 non-null    int64  
 8   chest hair                 541 non-null    int64  
 9   difficult to loose weight  541 non-null    int64  
 10  mood swings                541 non-null    int64  
 11  anxiety/depression/stress  541 non-null    int64  
 12  Irregular_sleep            541 non-null    int64  
 13  Weight gain(Y/N)           541 non-null    int64  

In [10]:
df.isnull().sum() # checking for null values

Sl No                        0
Patient File No              0
PCOS (Y/N)                   0
 Age (yrs)                   0
Pregnant(Y/N)                0
No of aborptions             0
Bloated                      0
facial hair                  0
chest hair                   0
difficult to loose weight    0
mood swings                  0
anxiety/depression/stress    0
Irregular_sleep              0
Weight gain(Y/N)             0
hair growth(Y/N)             0
Skin darkening (Y/N)         0
Hair loss(Y/N)               0
Pimples(Y/N)                 0
Fast food (Y/N)              0
Reg Exercise(Y/N)            0
Weight (Kg)                  0
Height(Cm)                   0
BMI                          0
Blood Group                  0
Pulse rate(bpm)              0
Cycle(months)                0
Cycle length(days)           0
Marriage Status (Yrs)        0
Hip(inch)                    0
Waist(inch)                  0
Waist/Hip Ratio              0
dtype: int64

In [11]:
df.describe() #statistical data for each numerical column

,Sl No,Patient File No,PCOS (Y/N),Age (yrs),Pregnant(Y/N),No of aborptions,Bloated,facial hair,chest hair,difficult to loose weight,...,Height(Cm),BMI,Blood Group,Pulse rate(bpm),Cycle(months),Cycle length(days),Marriage Status (Yrs),Hip(inch),Waist(inch),Waist/Hip Ratio
count,541.000000,541.000000,541.000000,541.000000,541.000000,541.000000,541.000000,541.000000,541.000000,541.000000,...,541.000000,541.000000,541.000000,541.000000,541.000000,541.00000,541.000000,541.000000,541.000000,541.000000
mean,271.000000,10271.000000,0.327172,31.430684,0.380776,0.288355,0.369686,0.321627,0.240296,0.380776,...,156.484835,24.311285,13.802218,73.247689,1.720887,4.94085,7.666728,37.992606,33.841035,0.891895
std,156.317519,156.317519,0.469615,5.411006,0.486027,0.692575,0.483166,0.467533,0.427658,0.486027,...,6.033545,4.056399,1.840812,4.430285,1.135989,1.49202,4.810733,3.967920,3.596894,0.046326
min,1.000000,10001.000000,0.000000,20.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,137.000000,12.417882,11.000000,13.000000,1.000000,0.00000,0.000000,26.000000,24.000000,0.755556
25%,136.000000,10136.000000,0.000000,28.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,152.000000,21.641274,13.000000,72.000000,1.000000,4.00000,4.000000,36.000000,32.000000,0.857143
50%,271.000000,10271.000000,0.000000,31.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,156.000000,24.238227,14.000000,72.000000,1.000000,5.00000,7.000000,38.000000,34.000000,0.894737
75%,406.000000,10406.000000,1.000000,35.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,...,160.000000,26.634958,15.000000,74.000000,2.000000,5.00000,10.000000,40.000000,36.000000,0.928571
max,541.000000,10541.000000,1.000000,48.000000,1.000000,5.000000,1.000000,1.000000,1.000000,1.000000,...,180.000000,38.900000,18.000000,82.000000,5.000000,12.00000,30.000000,48.000000,47.000000,0.979167


### dropping the unecessary columns

In [39]:
train = df.drop(columns = ["Sl No", "Patient File No", "PCOS (Y/N)"], axis = 1 )
#  adding inplace = true drops the columns from original dataframe,and if not used, it creates a new dataframe


      Age (yrs)  Pregnant(Y/N)  No of aborptions  Bloated  facial hair  \
0            28              0                 0        0            0   
1            36              1                 0        0            0   
2            33              1                 0        1            1   
3            37              0                 0        0            0   
4            25              1                 0        0            0   
..          ...            ...               ...      ...          ...   
536          35              0                 1        0            1   
537          30              1                 1        0            1   
538          36              0                 0        0            1   
539          27              0                 0        0            1   
540          23              0                 0        1            0   

     chest hair  difficult to loose weight  mood swings  \
0             0                          0          

In [38]:
df.columns


Index(['Sl No', 'Patient File No', 'PCOS (Y/N)', ' Age (yrs)', 'Pregnant(Y/N)',
       'No of aborptions', 'Bloated', 'facial hair', 'chest hair',
       'difficult to loose weight', 'mood swings', 'anxiety/depression/stress',
       'Irregular_sleep', 'Weight gain(Y/N)', 'hair growth(Y/N)',
       'Skin darkening (Y/N)', 'Hair loss(Y/N)', 'Pimples(Y/N)',
       'Fast food (Y/N)', 'Reg Exercise(Y/N)', 'Weight (Kg)', 'Height(Cm) ',
       'BMI', 'Blood Group', 'Pulse rate(bpm) ', 'Cycle(months)',
       'Cycle length(days)', 'Marriage Status (Yrs)', 'Hip(inch)',
       'Waist(inch)', 'Waist/Hip Ratio'],
      dtype='object')

### splitting numerical and categorical columns


In [14]:
numerical_cols = ["No of aborptions", "Weight (Kg)" ,"Height(Cm) " ,"BMI" ,"Blood Group" ,"Pulse rate(bpm) " ,"Cycle(months)" ,"Cycle length(days)" ,"Marriage Status (Yrs)" ,"Hip(inch)", "Waist(inch)", "Waist/Hip Ratio"]
categorical_cols = list(set(df.columns) - set(numerical_cols)- {"PCOS (Y/N)"})

In [15]:
numerical_cols


['No of aborptions',
 'Weight (Kg)',
 'Height(Cm) ',
 'BMI',
 'Blood Group',
 'Pulse rate(bpm) ',
 'Cycle(months)',
 'Cycle length(days)',
 'Marriage Status (Yrs)',
 'Hip(inch)',
 'Waist(inch)',
 'Waist/Hip Ratio']

In [16]:
categorical_cols

['Pimples(Y/N)',
 'difficult to loose weight',
 'Irregular_sleep',
 'Weight gain(Y/N)',
 'hair growth(Y/N)',
 'Bloated',
 'Pregnant(Y/N)',
 'chest hair',
 'anxiety/depression/stress',
 'mood swings',
 'Reg Exercise(Y/N)',
 'Skin darkening (Y/N)',
 'Fast food (Y/N)',
 'facial hair',
 ' Age (yrs)',
 'Hair loss(Y/N)']

### splitting data

In [45]:
df_train, df_test = train_test_split(train, test_size = 0.2, random_state = 42)

In [18]:
len(df_train), len(df_test)

(432, 109)

### scaling numerical columns
this is required because the numerical columns might have data 1 as well as 800. in this case 800 may be considered as outlier. so we need to apply standard scaling to bring all the values in a particular range 

In [19]:
scaler = StandardScaler()
def get_features_and_target_array(df, numerical_cols, categorical_cols, scaler):
  x_numeric_scaled = scaler.fit_transform(df[numerical_cols]) #it will take every data point of numerical column and it will transform it to -1 to 1 range
  x_categorical = df[categorical_cols].to_numpy() #categorical cols
  x = np.hstack((x_categorical, x_numeric_scaled)) #single array of the above two arrays so that it can directly be added to model
  y = df["PCOS (Y/N)"]
  return x, y

In [20]:
x_train, y_train = get_features_and_target_array(df_train, numerical_cols, categorical_cols, scaler)

### train
# 1. Logistic Regression

In [21]:
clf = LogisticRegression()
clf.fit(x_train, y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [22]:
x_test, y_test = get_features_and_target_array(df_test, numerical_cols, categorical_cols, scaler)

In [23]:
test_pred = clf.predict(x_test)

In [24]:
mean_squared_error(y_test, test_pred)

0.0

In [25]:
accuracy_score(y_test, test_pred)

1.0

In [26]:
confusion_matrix(y_test, test_pred)

array([[77,  0],
       [ 0, 32]])

# 2. Decision Tree

In [27]:
dc_clf = DecisionTreeClassifier()
dc_clf.fit(x_train, y_train)

dlf_pred = dc_clf.predict(x_test)
print(mean_squared_error(y_test, dlf_pred))
print(accuracy_score(y_test, dlf_pred))

0.0
1.0


# 3. SVM


In [47]:
svm_clf = SVC()
svm_clf.fit(x_train, y_train)
x_test = [[1,28,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,44,152,19.3,15,78,1,5,7,36,30,0.83333]]

svm_pred = svm_clf.predict(x_test)
print(svm_pred)
# print(mean_squared_error(y_test, svm_pred))
# print(accuracy_score(y_test, svm_pred))

[0]
